<a href="https://colab.research.google.com/github/ryunguo/LanguageDetector/blob/main/LanguageDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
import re
import spacy


In [44]:
df = pd.read_csv("Language Detection.csv")
type(df)

pandas.core.frame.DataFrame

In [49]:
language_list = ['English', 'French', 'Italian', 'Spanish', 'Portugese']
#(df.iloc[df['Language'].intersection(language_list)])
#df = [df['Language'] == x for x in language_list]
df = df.loc[df['Language'].isin(language_list)]
df


,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English
...,...,...
7807,"qual è stato il tuo errore, ti diamo da mangia...",Italian
7808,narcisa ha cambiato i suoi modi in un primo mo...,Italian
7809,Come' Il narcisismo di adesso Marian ha detto ...,Italian
7810,immagino che non vorrebbe più pane d'oro adess...,Italian


***Data Wrangling***

In [18]:
x = df['Text']
y = df['Language']

In [19]:
le = LabelEncoder()
y=le.fit_transform(y)

In [ ]:
data = []

for text in x:
  text = re.sub(r'[0-9]', '', text)
  text = re.sub(r'[\[\]]', '', text)

  text = text.lower()
  data.append(text)

data

In [ ]:

!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm
!python -m spacy download it_core_news_sm
!python -m spacy download pt_core_news_sm
!python -m spacy download es_core_news_sm



In [ ]:
nlp_en = spacy.load("en_core_web_sm")
nlp_fr = spacy.load("fr_core_news_sm")
nlp_it = spacy.load("it_core_news_sm")
nlp_pr = spacy.load("pt_core_news_sm")
nlp_sp = spacy.load("es_core_news_sm")

lang_dict = {
    "English": nlp_en,
    "French": nlp_fr,
    "Italian": nlp_it,
    "Portugese": nlp_pr,
    "Spanish": nlp_sp
    }

def removeNonsense(doc):
  return [token.lemma_ for token in doc if (not token.is_stop) and (not token.is_punct)]

for row in df.iterrows():
    doc = lang_dict[row[1][1]](row[1][0])
    print(removeNonsense(doc))

# for token in doc:
#     print(token)
#     print(f"\tPart-of-speech: {token.pos_}")
#     print(f"\tLemma: {token.lemma_}")
#     print(f"\tIs Stopword: {token.is_stop}")    
#     print(f"\tIs Punctuation: {token.is_punct}")